In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import backend, models, layers, optimizers, regularizers
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import re
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from string import punctuation
from nltk.corpus import wordnet as wn
import os
import sys

Next, we will import the data set.

In [ ]:
factual = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")
fakenews = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
bbc = pd.read_csv("/kaggle/input/bbcalldata3/BBCAll.csv")

In [ ]:
fakenews.head()

In [ ]:
factual.head()

In [ ]:
bbc.head()

In [ ]:
for key, count in fakenews.subject.value_counts().iteritems():
    print(f"{key}:\t{count}")
    
print(f"Total Records:\t{fakenews.shape[0]}")

for key, count in factual.subject.value_counts().iteritems():
    print(f"{key}:\t{count}")
    
print(f"Total Records:\t{factual.shape[0]}")

for key, count in bbc.subject.value_counts().iteritems():
    print(f"{key}:\t{count}")

print(f"Total Records:\t{bbc.shape[0]}")

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot("subject", data=fakenews)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot("subject", data=factual)
plt.show()

plt.figure(figsize=(10,5))
sns.countplot("subject", data=bbc)
plt.show()

In [ ]:
text = " ".join(str(each) for each in fakenews.text.unique())
wordcloud = WordCloud(
    max_words=200,
    colormap='Set3',
    background_color="black",
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
text = " ".join(str(each) for each in factual.text.unique())
wordcloud = WordCloud(
    max_words=200,
    colormap='Set3',
    background_color="black",
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
bbctext = " ".join(str(each) for each in bbc.text.unique())
wordcloud = WordCloud(
    max_words=200,
    colormap='Set3',
    background_color="black",
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(bbctext)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
factual['category'] = 1
fakenews['category'] = 0

In [ ]:
merged = pd.concat([factual,fakenews])

In [ ]:
sns.set_style('dark')
sns.countplot(merged.category)

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(x= 'subject', hue = 'category' , data=merged, palette = 'muted')
plt.show()

This graph shows us the data set is well balanced after its been merged.

In [ ]:
merged['text'] = merged['text']+ " " +merged['title']
del merged['title']
del merged['subject']
del merged['date']

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
merged.head()

Data Cleaning

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', ' ',text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

def clean_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

merged['text']=merged['text'].apply(clean_text)

In [ ]:
text = " ".join(str(each) for each in merged.text.unique())
wordcloud = WordCloud(
    max_words=200,
    colormap='Set3',
    background_color="black",
    stopwords = set(nltk.corpus.stopwords.words("english"))).generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud, interpolation='Bilinear')
plt.axis("off")
plt.figure(1,figsize=(12, 12))
plt.show()

In [ ]:
def strip_html(bbctext):
    soup = BeautifulSoup(bbctext, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(bbctext):
    return re.sub('\[[^]]*\]', '', bbctext)

def remove_between_square_brackets(bbctext):
    return re.sub(r'http\S+', ' ',bbctext)

def remove_stopwords(bbctext):
    bbcfinal_text = []
    for i in bbctext.split():
        if i.strip().lower() not in stop:
            bbcfinal_text.append(i.strip())
    return " ".join(bbcfinal_text)

def clean_bbctext(bbctext):
    bbctext = strip_html(bbctext)
    bbctext = remove_between_square_brackets(bbctext)
    bbctext = remove_stopwords(bbctext)
    return bbctext

bbc['text']=bbc['text'].apply(clean_bbctext)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(merged.text, merged.category, test_size= 0.15, random_state=0)

In [ ]:
xbbctrain, xbbctest, ybbctrain, ybbctest = train_test_split(bbc.text,bbc.subject, test_size= 0.15, random_state=0)

In [ ]:
maxword = 1000
length = 100

tokenizer = Tokenizer(num_words=maxword)
tokenizer.fit_on_texts(xtrain)

traintoken = tokenizer.texts_to_sequences(xtrain)
tokentest = tokenizer.texts_to_sequences(xtest)

X_train = sequence.pad_sequences(traintoken, maxlen =length)
X_test = sequence.pad_sequences(tokentest, maxlen =length)

tokenizer2 = Tokenizer(num_words=maxword)
tokenizer2.fit_on_texts(xbbctrain)

bbctraintoken = tokenizer2.texts_to_sequences(xbbctrain)
bbctokentest = tokenizer2.texts_to_sequences(xbbctest)

Xbbc_train = sequence.pad_sequences(bbctraintoken, maxlen =length)
Xbbc_test = sequence.pad_sequences(bbctokentest, maxlen =length)

In [ ]:
vocab_size = len(tokenizer.word_index)+1

embedding_dim = 100

In [ ]:
backend.clear_session()


model = models.Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length= length))
model.add(layers.Conv1D(128,kernel_size=5, activation ='relu'))
model.add(layers.Conv1D(128,kernel_size=5, activation ='relu'))
model.add(layers.Conv1D(128,kernel_size=5, activation ='relu'))
model.add(layers.Conv1D(64,kernel_size=5, activation ='relu'))
model.add(layers.Conv1D(64,kernel_size=5, activation ='relu'))
model.add(layers.Conv1D(64,kernel_size=5, activation ='relu'))
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation ='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile (optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

history = model.fit(X_train,
                    ytrain,
                    epochs = 15,
                    batch_size=500,
                    validation_data = (X_test, ytest),
                    verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

results = model.evaluate(X_test, ytest)
results

In [ ]:
pred = model.predict_classes(X_test)
pred[:5]

In [ ]:
confmatrix = confusion_matrix(ytest, pred)
confmatrix

In [ ]:
confmatrix = pd.DataFrame(confmatrix, index = ['FakeNews','Factual'], columns=['FakeNews','Factual'])
plt.figure(figsize = (10,10))
sns.heatmap(confmatrix, cmap='Reds', linecolor = 'black',
           linewidth = 1, annot= True, fmt='',xticklabels=['FakeNews','Factual'],
            yticklabels = ['FakeNews','Factual'])
plt.xlabel("Actual")
plt.ylabel("Predicted")

Now for GloVe

In [ ]:
base_dir = "/kaggle/input/glove6b/glove.6B.100d.txt"

max_seq_len = 100
max_num_words = 20000

In [ ]:
print('Indexing Word Vectors')
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(base_dir))

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
beds = np.stack(embeddings_index.values())
emean,estd = beds.mean(), beds.std()
esize = beds.shape[1]

windex = tokenizer.word_index
nb_words = min(max_num_words, len(windex))

embedding_matrix = embedding_matrix = np.random.normal(emean, estd, (nb_words, esize))
for word, i in windex.items():
    if i >= max_num_words: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
#Defining Neural Network
model2 = models.Sequential()
model2.add(layers.Embedding(max_num_words, output_dim=esize,
                    weights=[embedding_matrix], 
                    input_length=max_seq_len, trainable=False))
model2.add(layers.Dense(128, activation ='relu'))
model2.add(layers.Dense(128, activation ='relu'))
model2.add(layers.Dense(128, activation ='relu'))
model2.add(BatchNormalization())
model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation ='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.summary()

In [ ]:
model2.compile (optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

history = model2.fit(X_train,
                    ytrain,
                    epochs = 15,
                    batch_size=500,
                    validation_data = (X_test, ytest),
                    verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

results = model2.evaluate(X_test, ytest)
results

In [ ]:
pred2 = model2.predict_classes(X_test)
pred[:5]

In [ ]:
confmatrix2 = confusion_matrix(ytest, pred2)
confmatrix2

In [ ]:
confmatrix2 = pd.DataFrame(confmatrix2, index = ['FakeNews','Factual'], columns=['FakeNews','Factual'])
plt.figure(figsize = (10,10))
sns.heatmap(confmatrix2, cmap='Reds', linecolor = 'black',
           linewidth = 1, annot= True, fmt='',xticklabels=['FakeNews','Factual'],
            yticklabels = ['FakeNews','Factual'])
plt.xlabel("Actual")
plt.ylabel("Predicted")

Support Vector Machine

In [ ]:
fakenews['target'] = 'fake'
factual['target'] = 'true'
concat = pd.concat([factual,fakenews])

concat.head()

In [ ]:
concat = concat.drop(columns='category')
concat.head()

In [ ]:
x_svmtrain, x_svmtest, y_svmtrain, y_svmtest = train_test_split(concat['text'],concat.target, test_size=0.15, random_state=0)

In [ ]:
svmnews = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('svmmodel', LinearSVC())])

svmmodel = svmnews.fit(x_svmtrain, y_svmtrain)
svm_pred = svmnews.predict(x_svmtest)
print("accuracy: {}%".format(round(accuracy_score(y_svmtest, svm_pred)*100,2)))
print(classification_report(y_svmtest, svm_pred))

In [ ]:
nbnews = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('nbmodel', BernoulliNB())])

nbmodel = nbnews.fit(x_svmtrain, y_svmtrain)
nb_pred = nbnews.predict(x_svmtest)
print("accuracy: {}%".format(round(accuracy_score(y_svmtest, nb_pred)*100,2)))
print(classification_report(y_svmtest, nb_pred))

In [ ]:
lognews = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('logmodel', LogisticRegression())])

logmodel = lognews.fit(x_svmtrain, y_svmtrain)
log_pred = lognews.predict(x_svmtest)
print("accuracy: {}%".format(round(accuracy_score(y_svmtest, log_pred)*100,2)))
print(classification_report(y_svmtest, log_pred))